In [2]:
import matplotlib.pyplot as plt  # plotting
import csv

import numpy as np  # linear algebra
from scipy.io import loadmat
import glob
import mne
import pandas as pd

### Setup data information

In [62]:
emotion_label = {'0': 'neutral', '1': 'sad', '2': 'fear', '3': 'happy'}
session1_label = [1, 2, 3, 0, 2, 0, 0, 1, 0, 1, 2, 1, 1, 1, 2, 3, 2, 2, 3, 3, 0, 3, 0, 3]
session2_label = [2, 1, 3, 0, 0, 2, 0, 2, 3, 3, 2, 3, 2, 0, 1, 1, 2, 1, 0, 3, 0, 1, 3, 1]
session3_label = [1, 2, 2, 1, 3, 3, 3, 1, 1, 2, 1, 0, 2, 3, 3, 0, 2, 3, 0, 0, 2, 0, 1, 0]
target_channels = [15, 23, 24, 32, 33, 41]
channel_mapping = ['FT7', 'FT8', 'T7', 'T8', 'TP7', 'TP8']

# location = "../SEED_IV/eeg_raw_data"

In [63]:
location = 'eeg_raw_data/'
sessions = [sorted(glob.glob(location+"1/*.mat")), sorted(glob.glob(location+"2/*.mat")), sorted(glob.glob(location+"3/*.mat"))]
sessions_labels = [session1_label, session2_label, session3_label]
len(sessions[0][0])
sessions

[['eeg_raw_data/1/01.mat',
  'eeg_raw_data/1/02.mat',
  'eeg_raw_data/1/03.mat',
  'eeg_raw_data/1/04.mat',
  'eeg_raw_data/1/05.mat',
  'eeg_raw_data/1/06.mat',
  'eeg_raw_data/1/07.mat',
  'eeg_raw_data/1/08.mat',
  'eeg_raw_data/1/09.mat',
  'eeg_raw_data/1/10.mat',
  'eeg_raw_data/1/11.mat',
  'eeg_raw_data/1/12.mat',
  'eeg_raw_data/1/13.mat',
  'eeg_raw_data/1/14.mat',
  'eeg_raw_data/1/15.mat'],
 ['eeg_raw_data/2/01.mat',
  'eeg_raw_data/2/02.mat',
  'eeg_raw_data/2/03.mat',
  'eeg_raw_data/2/04.mat',
  'eeg_raw_data/2/05.mat',
  'eeg_raw_data/2/06.mat',
  'eeg_raw_data/2/07.mat',
  'eeg_raw_data/2/08.mat',
  'eeg_raw_data/2/09.mat',
  'eeg_raw_data/2/10.mat',
  'eeg_raw_data/2/11.mat',
  'eeg_raw_data/2/12.mat',
  'eeg_raw_data/2/13.mat',
  'eeg_raw_data/2/14.mat',
  'eeg_raw_data/2/15.mat'],
 ['eeg_raw_data/3/01.mat',
  'eeg_raw_data/3/02.mat',
  'eeg_raw_data/3/03.mat',
  'eeg_raw_data/3/04.mat',
  'eeg_raw_data/3/05.mat',
  'eeg_raw_data/3/06.mat',
  'eeg_raw_data/3/07.mat',

In [64]:
# Define basic data
ch_types = ['eeg'] * 6
sampling_freq = 1000

In [72]:
# Set counter
counter = {'0': 1, '1': 1, '2': 1, '3': 1}

# Select session to be generated
selected_session =  [1]
for s in selected_session:
    # Iterate all subjects in the session
    for idx, subjects in enumerate(sessions[s]):
       subject = loadmat(sessions[s][idx])

       # Iterate all trial
       for idx_l, t in enumerate(sessions_labels[s]):
            # Naming file
            label = emotion_label[str(sessions_labels[s][idx_l])]
            count = counter[str(t)]
            counter[str(t)] = counter[str(t)] + 1
            subject_name = list(subject.keys())[3]
            file_name = subject_name + '-' + str(label) + '-' + str(count) + '.csv'

            # Select data
            trial = subject[subject_name]

            # Select channel
            channel = trial[target_channels[0]]
            channel2 = trial[target_channels[1]]
            channel3 = trial[target_channels[2]]
            channel4 = trial[target_channels[3]]
            channel5 = trial[target_channels[4]]
            channel6 = trial[target_channels[5]]

            # Store raw data
            raw_data = np.array([channel, channel2, channel3, channel4, channel5, channel6])

            # Preprocess
            info = mne.create_info(ch_names=channel_mapping,
                               ch_types=ch_types,
                               sfreq=sampling_freq)
            simulated_raw = mne.io.RawArray(raw_data, info)
            FFTed_signal = simulated_raw.filter(1, 70., fir_design='firwin')
            resampled_signal = FFTed_signal.resample(150, npad="auto")
            times = np.array([resampled_signal.times]).T

            # Write file
            header = ['timestamps', 'FT7', 'FT8', 'T7', 'T8', 'TP7', 'TP8']
            render_data = np.hstack((times, resampled_signal.get_data().T))
            render_data = np.vstack((header, render_data))
            np.savetxt('output/' + file_name, render_data, delimiter=",", fmt='%s')


Creating RawArray with float64 data, n_channels=6, n_times=44201
    Range : 0 ... 44200 =      0.000 ...    44.200 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 70 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 70.00 Hz
- Upper transition bandwidth: 17.50 Hz (-6 dB cutoff frequency: 78.75 Hz)
- Filter length: 3301 samples (3.301 sec)

cz_eeg1-fear-1.csv
Creating RawArray with float64 data, n_channels=6, n_times=44201
    Range : 0 ... 44200 =      0.000 ...    44.200 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 70 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-pha